In [1]:
# [Student: Samuel] Import necessary libraries
import pandas as pd
import random

# [Student: Samuel] Set a seed
random.seed(123)

# [Student: Samuel] Define a pool of 30 unique supermarket items
item_pool = [
    'milk', 'bread', 'eggs', 'butter', 'cheese', 'apples', 'bananas', 'chicken', 'beef', 'rice',
    'pasta', 'sugar', 'salt', 'pepper', 'onions', 'tomatoes', 'lettuce', 'carrots', 'potatoes', 'cereal',
    'yogurt', 'juice', 'soda', 'coffee', 'tea', 'biscuits', 'chocolate', 'detergent', 'soap', 'toothpaste'
]

# [Student: Samuel] Set number of transactions to simulate
num_transactions = 3000

# [Student: Samuel] Generate a list of transactions
transactions = []
for _ in range(num_transactions):
    # Randomly choose 2 to 7 items per transaction
    num_items = random.randint(2, 7)
    transaction = random.sample(item_pool, num_items)
    transactions.append(transaction)

# [Student: Samuel] Convert list of transactions into a DataFrame for saving
df_transactions = pd.DataFrame({'Transaction': transactions})

# [Student: Samuel] Save simulated transactions to CSV
df_transactions.to_csv('supermarket_transactions.csv', index=False)

# [Student: Samuel] preview
df_transactions.head()

,Transaction
0,"[beef, eggs]"
1,"[beef, butter, chocolate, soap, detergent]"
2,"[salt, carrots]"
3,"[pasta, toothpaste, detergent, bread, apples, ..."
4,"[carrots, pasta, soda, chicken]"


### onehot Encoding

In [6]:
# Load the dataset
df_transactions = pd.read_csv('supermarket_transactions.csv')

In [7]:

# Create a set of all unique items across all transactions
unique_items = list(set(item for transaction in df_transactions['Transaction'] for item in eval(transaction)))

# Create a DataFrame for one-hot encoding
df_one_hot = pd.DataFrame(0, index=range(len(df_transactions)), columns=unique_items)

# Populate the one-hot DataFrame
for idx, transaction in enumerate(df_transactions['Transaction']):
    items = eval(transaction)  # Convert string representation of list to actual list
    for item in items:
        df_one_hot.loc[idx, item] = 1

# Preview the one-hot encoded DataFrame
df_one_hot.head()


,juice,bananas,apples,detergent,onions,tomatoes,yogurt,eggs,potatoes,biscuits,...,cheese,soda,bread,rice,beef,pasta,cereal,lettuce,tea,sugar
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,1,0,0,0,0,0,0,...,1,0,1,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0


#### Save Onehot encoded df

In [5]:
# Save the one-hot encoded DataFrame to a CSV
df_one_hot.to_csv('supermarket_one_hot_encoded.csv', index=False)
